In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
%matplotlib tk
# import pyvista as pv
# from pyvista import examples

### Inputs

##### constants

In [2]:
w = 0.0006 ## conversion constant to get sed fluxes in MT/yr
B = 1 ## lithology factor
pho_s = 1.7e3 ##sediment density in kg/m3

variates

In [3]:
T = 15 ## temperatures in °C, held constant
relief = np.append([0.2], np.arange(0.5, 5.5, 0.5)) # mountain relief in km
drainage_area = np.logspace(2, 7, num = 11) ##drainage area in m

In [4]:
# runoff = np.arange(0.1, 1.1, 0.1)*10e-3 ## runoff in km/yr
runoff = 0.5e-3

In [8]:
A, R = np.meshgrid(drainage_area, relief)
Qsy = w*B*T* (runoff*A)**0.31 * A**0.5 * R
Qs = Qsy/pho_s

### Define actual BQART formula

In [15]:
def bqart(w, B, T, runoff, area, relief):
    '''calculate sediment supply given inputs'''
    flag = input('Are you using precipitation or not:')
    if flag == 'runoff':
        rhos = 1.7e3 ##sediment density kgm3
        area_grid, relief_grid = np.meshgrid(area, relief)
        qsy = w*B*T* (runoff*area)**0.31 * area**0.5 * relief

        qs = qsy/rhos
    elif flag == 'discharge':
        print('potatoes')
    return qs

In [14]:
relief = np.append([0.2], np.arange(0.5, 5.5, 0.5)) # mountain relief in km
drainage_area = np.logspace(2, 7, num = 11) ##drainage area in m
Temp = 15
fig, ax = plt.subplots(figsize = (10,10))

## plot intial figure tobe manipulated

# fig, ax = plt.subplots()
cont = ax.contourf(A, R, Qs, cmap = 'gist_earth', levels = np.linspace(10e-5, 1, 10))

# plt.contourf(A, R, Qs, cmap = 'hsv', levels = np.linspace(10e-5, 1, 10))
fig.colorbar(cont, ax = ax)
ax.set_xlabel('Area, km2')
ax.set_ylabel('relief, km')

# adjust the main plot to make room for the sliders
fig.subplots_adjust(left=0.25, bottom=0.25)

# Make a horizontal slider to control the frequency.
axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
freq_slider = Slider(
    ax=axfreq,
    label='temperature, °C',
    valmin=2,
    valmax=40,
    valinit=Temp,
)

# Make a vertically oriented slider to control the amplitude
axamp = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
amp_slider = Slider(
    ax=axamp,
    label="Runoff",
    valmin=0.1e-3,
    valmax=1e-3,
    valinit=runoff,
    orientation="vertical"
)

## convert discharges to km3/yr 

# The function to be called anytime a slider's value changes
def update(val):
    cont = ax.contourf(A, R, bqart(w, B, freq_slider.val, amp_slider.val, A, R),
                       cmap = 'gist_earth', levels = np.linspace(10e-5, 1, 10))
    # cont.set_ydata(bqart(w, B, freq_slider.val, amp_slider.val, area, relief))
    fig.canvas.draw_idle()
    # fig.canvas.draw()


# register the update function with each slider
freq_slider.on_changed(update)
amp_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')


def reset(event):
    freq_slider.reset()
    amp_slider.reset()
button.on_clicked(reset)

plt.show()

In [16]:
Qs.max()# The parametrized function to be plotted
def f(t, amplitude, frequency):
    return amplitude * np.sin(2 * np.pi * frequency * t)

t = np.linspace(0, 1, 1000)

# Define initial parameters
init_amplitude = 5
init_frequency = 3

# Create the figure and the line that we will manipulate
fig, ax = plt.subplots()
line, = ax.plot(t, f(t, init_amplitude, init_frequency), lw=2)
ax.set_xlabel('Time [s]')

# adjust the main plot to make room for the sliders
fig.subplots_adjust(left=0.25, bottom=0.25)

# Make a horizontal slider to control the frequency.
axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
freq_slider = Slider(
    ax=axfreq,
    label='Frequency [Hz]',
    valmin=0.1,
    valmax=30,
    valinit=init_frequency,
)

# Make a vertically oriented slider to control the amplitude
axamp = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
amp_slider = Slider(
    ax=axamp,
    label="Amplitude",
    valmin=0,
    valmax=10,
    valinit=init_amplitude,
    orientation="vertical"
)


# The function to be called anytime a slider's value changes
def update(val):
    line.set_ydata(f(t, amp_slider.val, freq_slider.val))
    fig.canvas.draw_idle()


# register the update function with each slider
freq_slider.on_changed(update)
amp_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')


def reset(event):
    freq_slider.reset()
    amp_slider.reset()
button.on_clicked(reset)

plt.show()

1.1733951633167088